In [1]:
import requests
from html.parser import HTMLParser
from bs4 import BeautifulSoup
import json
import os
import argparse
import random
from pymystem3 import Mystem
from nltk.tokenize import sent_tokenize, word_tokenize
import stanza

ppln = stanza.Pipeline('ru', processors='tokenize,pos,lemma,depparse')

def get_article_by_ref(ref):
    response = requests.get(ref)
    soup = BeautifulSoup(response.content, 'lxml')
    res = dict()
    if response.status_code != 200:
        print('alarm', response.status_code)
        print(response)
        print(response.headers)
        print(response.content)
        print(ref)
        os.system('sleep 1')
        return None
    res['title'] = soup.find(class_="tm-article-snippet__title").get_text()
    res['content'] = soup.find(id='post-content-body').get_text()
    return res


def get_all_articles(url_prefix, pages_count=4):
    result = []
    iteration = 0
    for id_page in range(1, pages_count + 1):
        iteration += 1
        url = url_prefix + '/page' + str(id_page) + '/'
        response = requests.get(url)
        if response.status_code != 200:
            print("ALARM, ", response.status_code)
            print(response.content)
            print(response.headers)
            print(response.status_code)
            continue
        soup = BeautifulSoup(response.content, 'lxml')
        articles = soup.find_all(class_='tm-articles-list__item')
        for article in articles:
            snippets = article.find_all('a', class_='tm-article-snippet__title-link')
            if len(snippets) != 1:
                print(len(snippets), article)
                continue
            assert len(snippets) == 1
            snippet = snippets[0]
            id_state = article['id']
            res = dict()
            res['id'] = id_state
            res['page'] = url
            res['ref'] = 'https://habr.com' + snippet['href']
            result.append(res)
        if iteration % 5 == 0:
            os.system('sleep 1')
        if iteration % 10 == 0:
            print("processed=", iteration)

    return result


def get_articles_refs(filename):
    if not os.path.exists(filename):
        articles = get_all_articles('https://habr.com/ru/flows/popsci', 1000)
        print(len(articles))
        with open(filename, 'w+') as file:
            json.dump(articles, file)
        return articles
    with open(filename) as file:
        res = json.JSONDecoder().decode(file.read())
        return res


def load():
    filename = 'articles_content.json'
    articles = get_articles_refs('articles_refs_list.json')
    res = []
    iteration = 0
    for article in articles:
        iteration += 1
        tmp = get_article_by_ref(article['ref'])
        if tmp is None:
            continue
        article['title'] = tmp['title']
        article['content'] = tmp['content']
        res.append(article)
        if iteration % 30 == 0:
            print("iteration={}".format(iteration))
            os.system('sleep 1')
    with open(filename, 'w+') as file:
        json.dump(res, file)

2022-10-01 14:50:27 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2022-10-01 14:50:28 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |

2022-10-01 14:50:28 INFO: Use device: cpu
2022-10-01 14:50:28 INFO: Loading: tokenize
2022-10-01 14:50:28 INFO: Loading: pos
2022-10-01 14:50:28 INFO: Loading: lemma
2022-10-01 14:50:28 INFO: Loading: depparse
2022-10-01 14:50:29 INFO: Done loading processors!


In [378]:
load()

iteration=30
iteration=60
iteration=90
iteration=120
iteration=150
iteration=180
iteration=210
iteration=240
iteration=270
iteration=300
iteration=330
iteration=360
iteration=390
iteration=420
iteration=450
iteration=480
iteration=510
iteration=540
iteration=570
iteration=600
iteration=630
iteration=660
iteration=690
iteration=720
iteration=750
iteration=780


In [2]:
import pandas as pd
df = pd.read_json('/Users/danialaliev/testgit/articles_content.json')

In [3]:
df

,id,page,ref,title,content
0,684134,https://habr.com/ru/flows/popsci/page1/,https://habr.com/ru/company/selectel/blog/684134/,Все идет не по плану: цены на SSD продолжают п...,"\r\nВ начале августа мы писали о том, что стои..."
1,688770,https://habr.com/ru/flows/popsci/page1/,https://habr.com/ru/post/688770/,Имитационные тренажеры и импортозамещение Hone...,Компания Honeywell решила свернуть свой сущест...
2,688764,https://habr.com/ru/flows/popsci/page1/,https://habr.com/ru/post/688764/,Как сделали 9 NLU ботов за 5 дней с интеграция...,Краткое описание проектаВ данном проекте перед...
3,688752,https://habr.com/ru/flows/popsci/page1/,https://habr.com/ru/post/688752/,Целеустремленный зомби,"Давние читатели “crawl” могут вспомнить, что у..."
4,688710,https://habr.com/ru/flows/popsci/page1/,https://habr.com/ru/post/688710/,"Человек, маска и аватара. Не рано ли переходит...",В эпоху всеобщей виртуальности и общения через...
...,...,...,...,...,...
795,679366,https://habr.com/ru/flows/popsci/page40/,https://habr.com/ru/company/agima/blog/679366/,«У спорта как будто плохой маркетинг: люди дум...,"Сотрудники семи IT-компаний рассказывают, заче..."
796,679382,https://habr.com/ru/flows/popsci/page40/,https://habr.com/ru/post/679382/,Исследования: как мозг строит гипотезы об окру...,Некоторые нейробиологи объясняют восприятие че...
797,679288,https://habr.com/ru/flows/popsci/page40/,https://habr.com/ru/company/leader-id/blog/679...,Как устроен портативный электрохимический гене...,"Речь пойдет про портативную электростанцию, ко..."
798,679370,https://habr.com/ru/flows/popsci/page40/,https://habr.com/ru/post/679370/,Компания Meta* призывает отказаться от високос...,\r\nПонятие дополнительной (високосной) секунд...


In [5]:
# Уберем из текстов специальные символы

def rm_n_r(s):
    s = s.replace('\n', '.')
    s = s.replace('\r', ' ')
    return " ".join(s.split())

df.content = df.content.apply(rm_n_r)

In [26]:
m = Mystem()

def lemmatize(text):
    sents = text.split(' ')
    pr_text = m.analyze(text)
    tokens = [x['text'] for x in pr_text]
    lemmas = m.lemmatize(text)
    return ' '.join(tokens), ' '.join(lemmas)

In [104]:
#Постмотрим на случайное предложение и попытаемся проанализировать его 
text = random.choice(df.content)
sent = random.choice(text.split('.'))
n_sent, lem_sent = lemmatize(sent)
doc = ppln(n_sent)
print(*[f'word: {word.text}\t{word.upos}\tfeats\thead: {snt.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for snt in doc.sentences for word in snt.words], sep='\n')
print(sent, '---->', lem_sent)

# Как видим, все части речи  и лемматизации идентифицированы моделью верно
# head для каждого word означает слово, от которого оно зависит, т.е.:
# Сферами (какими?) дополнительными ----> сферами является главным словом, а дополнительными - зависимым
# Тип связи между ними - согласование

#Сферами (чего?) применения -> сферами - главное, применения - зависимое
#тип связи - управление

# все зависимости разбирать не будем - обратим еще внимание только на ошибочную интерпретацию предложения - слова
# 'использование', 'генерация', 'мониторинг', следуя обработке, зависят от 'промышленность', что очевидно не так - 
# - все эти слова являются однородными членами предложения 

word: Дополнительными	ADJ	feats	head: сферами	deprel: amod
word: сферами	NOUN	feats	head: называют	deprel: obl
word: применения	NOUN	feats	head: сферами	deprel: nmod
word: устройства	NOUN	feats	head: применения	deprel: nmod
word: ,	PUNCT	feats	head: воды	deprel: punct
word: кроме	ADP	feats	head: воды	deprel: case
word: питьевой	ADJ	feats	head: воды	deprel: amod
word: воды	NOUN	feats	head: сферами	deprel: nmod
word: ,	PUNCT	feats	head: сферами	deprel: punct
word: называют	VERB	feats	head: root	deprel: root
word: пищевую	ADJ	feats	head: промышленность	deprel: amod
word: промышленность	NOUN	feats	head: называют	deprel: obj
word: ,	PUNCT	feats	head: использование	deprel: punct
word: вторичное	ADJ	feats	head: использование	deprel: amod
word: использование	NOUN	feats	head: промышленность	deprel: conj
word: сточных	ADJ	feats	head: вод	deprel: amod
word: вод	NOUN	feats	head: использование	deprel: nmod
word: ,	PUNCT	feats	head: генерация	deprel: punct
word: порциональная	ADJ	feats	head: генерац

In [7]:
#Пример неправильной лемматизации грамматических омонимов
# Пила в первом случае правильно принята за сущетсвительное, однако во втором - за глагол, из-зач чего рушатся синтаксические связи
sent = ' '.join(df.content[214].split())
n_sent, lem_sent = lemmatize(sent)
for x, y in zip(n_sent.split('.'), lem_sent.split('.')):
    if ' пила ' in x:
        doc = ppln(x)
        print(*[f'word: {word.text}\t{word.upos}\tfeats\thead: {snt.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for snt in doc.sentences for word in snt.words], sep='\n')
        print(x, '---->', y)

word: Это	PRON	feats	head: может	deprel: nsubj
word: может	VERB	feats	head: root	deprel: root
word: быть	AUX	feats	head: цеп	deprel: cop
word: цеп	NOUN	feats	head: может	deprel: obl
word: ,	PUNCT	feats	head: диск	deprel: punct
word: диск	NOUN	feats	head: цеп	deprel: conj
word: ,	PUNCT	feats	head: лезвие	deprel: punct
word: лезвие	NOUN	feats	head: цеп	deprel: conj
word: ,	PUNCT	feats	head: молот	deprel: punct
word: молот	NOUN	feats	head: цеп	deprel: conj
word: ,	PUNCT	feats	head: пила	deprel: punct
word: циркулярная	ADJ	feats	head: пила	deprel: amod
word: пила	NOUN	feats	head: цеп	deprel: conj
word: или	CCONJ	feats	head: пила	deprel: cc
word: ,	PUNCT	feats	head: или	deprel: punct
word: например	ADV	feats	head: пила	deprel: parataxis
word: ,	PUNCT	feats	head: например	deprel: punct
word: кольцевая	ADJ	feats	head: пила	deprel: nsubj
word: пила	VERB	feats	head: может	deprel: conj
word: по	ADP	feats	head: периметру	deprel: case
word: всему	DET	feats	head: периметру	deprel: det
word: перимет

In [27]:
#Пример ошибочной и грамотной обработки потенциальной ошибки, связанной с грамматическим омонимом 'течь'
for sent in df.content.tolist():
    n_sent, lem_sent = lemmatize(sent)
    for x, y in zip(n_sent.split('.'), lem_sent.split('.')):
        if ' течь ' in x:
            doc = ppln(x)
            print(*[f'word: {word.text}\t{word.upos}\tfeats\thead: {snt.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for snt in doc.sentences for word in snt.words], sep='\n')
            print(x, '---->', y)
            break

word: Получается	VERB	feats	head: root	deprel: root
word: ситуация	NOUN	feats	head: Получается	deprel: nsubj
word: с	ADP	feats	head: лодкой	deprel: case
word: дырявой	ADJ	feats	head: лодкой	deprel: amod
word: лодкой	NOUN	feats	head: ситуация	deprel: nmod
word: :	PUNCT	feats	head: заткнули	deprel: punct
word: в	ADP	feats	head: месте	deprel: case
word: одном	NUM	feats	head: месте	deprel: nummod
word: месте	NOUN	feats	head: заткнули	deprel: obl
word: заткнули	VERB	feats	head: лодкой	deprel: parataxis
word: течь	VERB	feats	head: заткнули	deprel: xcomp
word: ,	PUNCT	feats	head: открылась	deprel: punct
word: а	CCONJ	feats	head: открылась	deprel: cc
word: в	ADP	feats	head: другом	deprel: case
word: другом	ADJ	feats	head: открылась	deprel: obl
word: она	PRON	feats	head: открылась	deprel: nsubj
word: открылась	VERB	feats	head: заткнули	deprel: conj
  Получается   ситуация   с   дырявой   лодкой :  в   одном   месте   заткнули   течь ,  а   в   другом   она   открылась  ---->   получаться   ситу

In [92]:
#Пару примеров лексических омонимов, которые, однако, никак не влияют на корретность анализа предложения
i = 0

for sent in df.content:
    n_sent, lem_sent = lemmatize(sent)
    for x, y in zip(n_sent.split('.'), lem_sent.split('.')):
        if ' замок ' in x or ' коса ' in x or 'кисть ' in x:
            doc = ppln(x)
            print(*[f'word: {word.text}\t{word.upos}\tfeats\thead: {snt.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for snt in doc.sentences for word in snt.words], sep='\n')
            print(x, '---->', y)
            i += 1
    if i > 10:
        break

word: Например	ADV	feats	head: кольцо	deprel: parataxis
word: ,	PUNCT	feats	head: Например	deprel: punct
word: кольцо	NOUN	feats	head: root	deprel: root
word: с	ADP	feats	head: огнетушителем	deprel: case
word: огнетушителем	NOUN	feats	head: кольцо	deprel: nmod
word: ,	PUNCT	feats	head: банки	deprel: punct
word: сложно	ADV	feats	head: открывающиеся	deprel: advmod
word: открывающиеся	VERB	feats	head: банки	deprel: acl
word: банки	NOUN	feats	head: кольцо	deprel: conj
word: с	ADP	feats	head: лекарствами	deprel: case
word: лекарствами	NOUN	feats	head: банки	deprel: nmod
word: ,	PUNCT	feats	head: замок	deprel: punct
word: сейфовый	ADJ	feats	head: замок	deprel: amod
word: замок	NOUN	feats	head: кольцо	deprel: conj
word: ,	PUNCT	feats	head: системы	deprel: punct
word: системы	NOUN	feats	head: кольцо	deprel: conj
word: сигнализации	NOUN	feats	head: системы	deprel: nmod
word: …	PUNCT	feats	head: кольцо	deprel: punct
word: Нам	PRON	feats	head: нужно	deprel: iobj
word: ведь	PART	feats	head: нужно	

In [24]:
# Эллипсис
sent = ' '.join(df.content[5].split())
n_sent, lem_sent = lemmatize(sent)
for x, y in zip(n_sent.split('.'), lem_sent.split('.')):
    if 'именно' in x:
        doc = ppln(x)
        print(*[f'word: {word.text}\t{word.upos}\tfeats\thead: {snt.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for snt in doc.sentences for word in snt.words], sep='\n')
        print(x, '---->', y)
        break

word: А	CCONJ	feats	head: старт	deprel: cc
word: именно	PART	feats	head: А	deprel: fixed
word: —	PUNCT	feats	head: А	deprel: punct
word: внятный	ADJ	feats	head: старт	deprel: amod
word: ,	PUNCT	feats	head: однозначный	deprel: punct
word: однозначный	ADJ	feats	head: внятный	deprel: conj
word: ,	PUNCT	feats	head: четкий	deprel: punct
word: четкий	ADJ	feats	head: внятный	deprel: conj
word: старт	NOUN	feats	head: root	deprel: root
  А   именно  —  внятный ,  однозначный ,  четкий   старт  ---->   а   именно  —  внятный ,  однозначный ,  четкий   старт 


In [57]:
#Референциальные цепочки
sent = ' '.join(df.content[6].split())
n_sent, lem_sent = lemmatize(sent)
for i in [50, 51]:
    x, y = n_sent.split('.')[i], lem_sent.split('.')[i]
    doc = ppln(x)
    print(*[f'word: {word.text}\t{word.upos}\tfeats\thead: {snt.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for snt in doc.sentences for word in snt.words], sep='\n')
    print(x, '---->', y)

word: Естественно	ADV	feats	head: разрабатывал	deprel: parataxis
word: ,	PUNCT	feats	head: Естественно	deprel: punct
word: Советский	ADJ	feats	head: Союз	deprel: amod
word: Союз	PROPN	feats	head: разрабатывал	deprel: nsubj
word: разрабатывал	VERB	feats	head: root	deprel: root
word: собственные	ADJ	feats	head: аппараты	deprel: amod
word: аппараты	NOUN	feats	head: разрабатывал	deprel: obj
 Естественно ,  Советский   Союз   разрабатывал   собственные   аппараты  ---->  естественно ,  советский   союз   разрабатывать   собственный   аппарат 
word: В	ADP	feats	head: 70	deprel: case
word: 70	NUM	feats	head: установлены	deprel: nummod
word: -	PUNCT	feats	head: х	deprel: punct
word: х	NUM	feats	head: 70	deprel: nummod
word: они	PRON	feats	head: установлены	deprel: nsubj:pass
word: были	AUX	feats	head: установлены	deprel: aux:pass
word: установлены	VERB	feats	head: root	deprel: root
word: в	ADP	feats	head: городах	deprel: case
word: крупнейших	ADJ	feats	head: городах	deprel: amod
word: городах	

In [58]:
sent = ' '.join(df.content[7].replace('тарифДля', 'тариф. Для').split())
n_sent, lem_sent = lemmatize(sent)
for i in [71, 72]:
    x, y = n_sent.split('.')[i], lem_sent.split('.')[i]
    doc = ppln(x)
    print(*[f'word: {word.text}\t{word.upos}\tfeats\thead: {snt.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for snt in doc.sentences for word in snt.words], sep='\n')
    print(x, '---->', y)

word: Для	ADP	feats	head: расчета	deprel: case
word: расчета	NOUN	feats	head: существует	deprel: obl
word: любой	DET	feats	head: перевозки	deprel: det
word: перевозки	NOUN	feats	head: расчета	deprel: nmod
word: по	ADP	feats	head: дороге	deprel: case
word: железной	ADJ	feats	head: дороге	deprel: amod
word: дороге	NOUN	feats	head: существует	deprel: obl
word: существует	VERB	feats	head: root	deprel: root
word: специальный	ADJ	feats	head: справочник	deprel: amod
word: справочник	NOUN	feats	head: существует	deprel: nsubj
word: ,	PUNCT	feats	head: утвержденный	deprel: punct
word: утвержденный	VERB	feats	head: справочник	deprel: acl
word: на	ADP	feats	head: уровне	deprel: case
word: государственном	ADJ	feats	head: уровне	deprel: amod
word: уровне	NOUN	feats	head: утвержденный	deprel: obl
word: –	PUNCT	feats	head: Прейскурант	deprel: punct
word: Прейскурант	PROPN	feats	head: уровне	deprel: parataxis
word: №	SYM	feats	head: Прейскурант	deprel: nummod:entity
word: 10	NUM	feats	head: №	deprel: n